In [143]:
import os
import glob
import oommfodt
import numpy as np
import traitlets
import widgets
import ipywidgets
import matplotlib.pyplot as plt
import IPython.display
from IPython.html import widgets
from IPython.display import display, clear_output

ImportError: No module named 'widgets'

In [212]:
class DataAnalysisODT:
    def __init__(self, name):
        self.dirname = os.path.join(name, "")
        
        odtfile = max(glob.iglob("{}*.odt".format(self.dirname)),
                      key=os.path.getctime)
        self.dt = oommfodt.OOMMFodt(odtfile).df
        self.time = self.dt["t"].as_matrix()/1e-9
        
        self.slider = ipywidgets.FloatRangeSlider(value=[self.time[0], self.time[-1]],
                                                  min=self.time[0], 
                                                  max=self.time[-1],
                                                  step=self.time[1]-self.time[0],
                                                  description="t (ns):",
                                                  readout=True,
                                                  layout=ipywidgets.Layout(width="100%"))

        self.select = ipywidgets.Select(options=list(self.dt.columns.values),
                                        description="Output:")
        self.slider.observe(self.update_output_slider)
        self.select.observe(self.update_output_slider)
        self.out = widgets.Output(layout=ipywidgets.Layout(width="300%"))
    
        self.update_output_slider(None)
        
    def _ipython_display_(self):
        box1 = ipywidgets.VBox([self.slider, self.out])
        box2 = ipywidgets.HBox([self.select, box1])
        display(box2)
            
    def update_output_slider(self, value):
        self.out.clear_output(wait=True)
        plt.close()
        plt.plot(self.time, self.dt[self.select.value])
        plt.xlabel("t (ns)")
        plt.ylabel(self.select.value)
        plt.xlim([self.slider.value[0], self.slider.value[1]])
        plt.grid()
        with self.out:
            display(plt.gcf())

In [213]:
d = Data()
da = DataAnalysisODT(name="stdprob4")
da